In [ ]:
!git clone https://github.com/jee51/tabata.git

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import re

In [2]:
# Chargement des données.
datadir = "../Data/cleaned/"
final_file = datadir + "AFL1EB_cleaned_final.h5"
norm_file = datadir + "AFL1EB_normed_alt.h5"

ds = Opset(final_file)

NameError: name 'Opset' is not defined

In [ ]:
def extract_phase_features(df, change_points):
    """
    Extrait les caractéristiques de chaque phase de vol.
    
    Arguments :
        df : pd.DataFrame - Le DataFrame contenant les données de vol.
        change_points : list - Liste des points de changement de phase.
        
    Returns :
        features : list - Liste des caractéristiques extraites pour chaque phase.
    """
    features = []
    for i in range(len(change_points) - 1):
        t0, t1 = change_points[i], change_points[i + 1]
        phase_data = df.iloc[t0:t1]
        
        # Moyenne et variance de l'altitude
        mean_altitude = phase_data['ALT[m]'].mean()
        var_altitude = phase_data['ALT[m]'].var()

        # Moyenne et variance de la vitesse de montée/descente
        d_altitude = np.gradient(phase_data['ALT[m]'].values)
        mean_rate_of_climb = d_altitude.mean()
        var_rate_of_climb = d_altitude.var()

        # Ajouter ces caractéristiques dans la liste
        features.append([mean_altitude, var_altitude, mean_rate_of_climb, var_rate_of_climb])
    
    return features

def apply_kmeans_clustering(features, n_clusters=3):
    """
    Applique l'algorithme K-Means pour regrouper les phases en clusters.
    
    Arguments :
        features : list - Liste des caractéristiques des phases de vol.
        n_clusters : int - Nombre de clusters à créer.
        
    Returns :
        kmeans.labels_ : ndarray - Les étiquettes des clusters pour chaque phase.
    """
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    kmeans.fit(features)
    return kmeans.labels_

def visualize_clusters(features, labels):
    """
    Visualise les clusters en utilisant une réduction de dimension (PCA).
    
    Arguments :
        features : list - Liste des caractéristiques des phases de vol.
        labels : ndarray - Les étiquettes des clusters.
    """
    # Réduction de dimension avec PCA
    pca = PCA(n_components=2)
    reduced_features = pca.fit_transform(features)

    # Plot des clusters
    plt.figure(figsize=(8, 6))
    plt.scatter(reduced_features[:, 0], reduced_features[:, 1], c=labels, cmap='viridis')
    plt.title('Clustering des Phases de Vol')
    plt.xlabel('Composante principale 1')
    plt.ylabel('Composante principale 2')
    plt.colorbar()
    plt.show()

In [ ]:




# Exemple d'utilisation :
# Supposons que 'ds' est votre ensemble de données de vol (une liste de DataFrames)
# et que nous avons déjà détecté les points de changement de phase pour chaque vol.

n_clusters = 3  # Nombre de clusters à détecter

# Extraction des caractéristiques pour chaque vol
all_features = []
all_labels = []

for df in ds:
    change_points = compute_points(df, n_points=6, method="ruptures", col="ALT[m]")
    features = extract_phase_features(df, change_points)
    labels = apply_kmeans_clustering(features, n_clusters=n_clusters)
    
    all_features.extend(features)
    all_labels.extend(labels)

# Visualisation des résultats
visualize_clusters(all_features, all_labels)